## Table of Contents

* Data Set
* Defining a Baseline Model
* Introduction to Deep Neural Networks
    * Introducing Keras
    * First Keras Model
* What Is a Word Embedding?
     * One-Hot Encoding
     * Word Embeddings
     * Keras Embedding Layer
* Convolutional Neural Networks (CNN)
*  Hyperparameters Optimization

### Importing Packages

In [176]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier
import os



Extract the folder into a data folder and go ahead and load the data with Pandas:

In [177]:
import pandas as pd

filepath_dict = {'yelp':   'yelp_labelled.txt',
                 'amazon': 'amazon_cells_labelled.txt',
                 'imdb':   'imdb_labelled.txt'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)
# df_list

In [178]:
df = pd.concat(df_list)


In [ ]:
df['source']

In [179]:
df = pd.read_csv("nlp_casestudy1_Data.csv")

In [181]:
df.head(5)

,Unnamed: 0,sentence,label,source
0,0,Wow... Loved this place.,1,yelp
1,1,Crust is not good.,0,yelp
2,2,Not tasty and the texture was just nasty.,0,yelp
3,3,Stopped by during the late May bank holiday of...,1,yelp
4,4,The selection on the menu was great and so wer...,1,yelp


In [14]:
df.to_csv("nlp_casestudy1_Data.csv")

Now use the `CountVectorizer` provided by the `scikit-learn` library to vectorize sentences. It takes the words of each sentence and creates a vocabulary of all the unique words in the sentences. This vocabulary can then be used to create a feature vector of the count of the words:

In [ ]:
#vectorization : converting raw text documnts into meaningful vectors (real numbers/ints)

In [184]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [185]:
sentences = ['Rashmi likes ice cream', 'Rashmi hates chocolate.','Rashmi is absent today']

In [234]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document Dog dog dog dog.',
    'This document is the second second  Dog document.',
    'And this is the third one dog.',
    'Is this the first document dog?',
]
vectorizer = CountVectorizer(lowercase=True)
X = vectorizer.fit_transform(corpus)
print(len(vectorizer.get_feature_names()))
print(vectorizer.vocabulary_)





10
{'this': 9, 'is': 4, 'the': 7, 'first': 3, 'document': 1, 'dog': 2, 'second': 6, 'and': 0, 'third': 8, 'one': 5}


In [235]:
Tf-idf vectorizer : 

'This is the first document Dog dog dog dog.'

In [236]:
X.toarray()[0]

array([0, 1, 4, 1, 1, 0, 0, 1, 0, 1], dtype=int64)

In [ ]:
Tfidf,bow,word embeddings,onehot encoding-->

In [237]:
# Extracting features from text files
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vect = TfidfVectorizer()

In [241]:
sentences[0]

'sss likes ice cream'

In [244]:
tf_vect.fit_transform(sentences).toarray()[0]

array([0.        , 0.        , 0.54645401, 0.        , 0.54645401,
       0.        , 0.32274454, 0.        , 0.54645401, 0.        ])

In [238]:
sentences = ['sss likes ice cream', 'rashmi hates likes chocolate','rashmi is likes absent today']

In [212]:
vectorizer = CountVectorizer(min_df=0,lowercase=False)
vect_sent=vectorizer.fit_transform(sentences)


In [213]:
vectorizer.vocabulary_

{'sss': 8,
 'likes': 6,
 'ice': 4,
 'cream': 2,
 'rashmi': 7,
 'hates': 3,
 'chocolate': 1,
 'is': 5,
 'absent': 0,
 'today': 9}

## Defining a Baseline Model

First, you are going to split the data into a training and testing set which will allow you to evaluate the accuracy and see if your model generalizes well. This means whether the model is able to perform well on data it has not seen before. This is a way to see if the model is overfitting.

**Overfitting** is when a model is trained too well on the training data. You want to avoid overfitting, as this would mean that the model mostly just memorized the training data. This would account for a large accuracy with the training data but a low accuracy in the testing data. 

In [ ]:
# ways of reducing overfitting:
1.reduce the complexity of the model.
2.dropout() layer to remove some percentage of the connections
3L1/L2 regularization. ?

In [ ]:
df

We start by taking the Yelp data set which we extract from our concatenated data set. From there, we take the sentences and labels.

In [29]:
df_yelp = df[df['source'] == 'yelp']
sentences = df_yelp['sentence'].values
y = df_yelp['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

Create the feature vectors for each sentence of the training and testing set:

In [32]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

In [38]:
len(vectorizer.vocabulary_)

1714

In [35]:
X_train.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

`CountVectorizer` performs tokenization which separates the sentences into a set of **tokens**. It additionally removes punctuation and special characters and can apply other preprocessing to each word. If you want, you can use a custom tokenizer from the NLTK library with the CountVectorizer or use any number of the customizations which you can explore to improve the performance of your model.

The classification model we are going to use is the logistic regression which is a simple yet powerful linear model that is mathematically speaking in fact a form of regression between 0 and 1 based on the input feature vector. By specifying a cutoff value (by default 0.5), the regression model is used for classification. 

In [47]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)




LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [49]:
y_pred = classifier.predict(X_test)

In [50]:
accuracy_score(y_test,y_pred)

0.7486631016042781

In [51]:
confusion_matrix(y_test,y_pred)

array([[67, 29],
       [18, 73]], dtype=int64)

In [53]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.70      0.74        96
           1       0.72      0.80      0.76        91

    accuracy                           0.75       187
   macro avg       0.75      0.75      0.75       187
weighted avg       0.75      0.75      0.75       187



You can see that the logistic regression reached an impressive 79.6%, but let’s have a look how this model performs on the other data sets that we have. In this script, we perform and evaluate the whole process for each data set that we have:

In [44]:
 df['source'].unique()

array(['yelp', 'amazon', 'imdb'], dtype=object)

In [54]:
for source in df['source'].unique():
    df_source = df[df['source'] == source]
    sentences = df_source['sentence'].values
    y = df_source['label'].values

    sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentences, y, test_size=0.25, random_state=1000)

    vectorizer = CountVectorizer()
    vectorizer.fit(sentences_train)
    X_train = vectorizer.transform(sentences_train)
    X_test  = vectorizer.transform(sentences_test)

    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    print('Accuracy for {} data: {:.4f}'.format(source, score))

Accuracy for yelp data: 0.7960
Accuracy for amazon data: 0.7960
Accuracy for imdb data: 0.7487


C:\Users\7314315\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
# try naive bayes/decision tree/random foresr/knn/ k means(unsupervissed)

Great! You can see that this fairly simple model achieves a fairly good accuracy.

## Introduction to Deep Neural Networks
**Neural networks**, or sometimes called **artificial neural network (ANN)** or**feedforward neural network**, are computational networks which were vaguely inspired by the neural networks in the human brain. They consist of neurons (also called nodes) which are connected like in the graph below.

You start by having a layer of input neurons where you feed in your feature vectors and the values are then feeded forward to a hidden layer. At each connection, you are feeding the value forward, while the value is multiplied by a weight and a bias is added to the value. This happens at every connection and at the end you reach an output layer with one or more output nodes.

If you want to have a binary classification you can use one node, but if you have multiple categories you should use multiple nodes for each category:

### Introducing Keras

Keras is a deep learning and neural networks API by François Chollet which is capable of running on top of Tensorflow (Google), Theano or CNTK (Microsoft). To quote the wonderful book by François Chollet, Deep Learning with Python:

    Keras is a model-level library, providing high-level building blocks for developing deep-learning models. It doesn’t handle low-level operations such as tensor manipulation and differentiation. Instead, it relies on a specialized, well-optimized tensor library to do so, serving as the backend engine of Keras (Source)

It is a great way to start experimenting with neural networks without having to implement every layer and piece on your own. For example Tensorflow is a great machine learning library, but you have to implement a lot of boilerplate code to have a model running.

### First Keras Model

Keras supports two main types of models. You have the Sequential model API and the functional API which can do everything of the Sequential model but it can be also used for advanced models with complex network architectures.

The Sequential model is a linear stack of layers, where you can use the large variety of available layers in Keras. The most common layer is the Dense layer which is your regular densely connected neural network layer with all the weights and biases that you are already familiar with.

Before we build our model, we need to know the input dimension of our feature vectors. This happens only in the first layer since the following layers can do automatic shape inference. In order to build the Sequential model, you can add layers one by one in order

In [61]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(512, input_dim=input_dim, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, input_dim=input_dim, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(100, input_dim=input_dim, activation='relu'))
model.add(layers.Dropout(0.2))


model.add(layers.Dense(1, activation='sigmoid'))

In [62]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               1283072   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               25700     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                

In [68]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Train on 561 samples, validate on 187 samples
Epoch 1/100
561/561 [==============================] - 2s 3ms/step - loss: 1.3529e-04 - accuracy: 1.0000 - val_loss: 1.5805 - val_accuracy: 0.7647
Epoch 2/100
561/561 [==============================] - 2s 3ms/step - loss: 1.3054e-04 - accuracy: 1.0000 - val_loss: 1.6162 - val_accuracy: 0.7594
Epoch 3/100
561/561 [==============================] - 2s 3ms/step - loss: 1.4214e-04 - accuracy: 1.0000 - val_loss: 1.6510 - val_accuracy: 0.7594
Epoch 4/100
561/561 [==============================] - 2s 3ms/step - loss: 1.1777e-04 - accuracy: 1.0000 - val_loss: 1.6645 - val_accuracy: 0.7594
Epoch 5/100
561/561 [==============================] - 2s 3ms/step - loss: 7.0555e-05 - accuracy: 1.0000 - val_loss: 1.6752 - val_accuracy: 0.7594
Epoch 6/100
561/561 [==============================] - 2s 3ms/step - loss: 9.7323e-05 - accuracy: 1.0000 - val_loss: 1.6765 - val_accuracy: 0.7647
Epoch 7/100
561/561 [==============================] - 2s 3ms/step - los

KeyboardInterrupt: 

In [69]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 1.0000
Testing Accuracy:  0.7380


In [70]:
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [71]:
plot_history(history)

NameError: name 'history' is not defined

## What Is a Word Embedding?

Text is considered a form of sequence data similar to time series data that you would have in weather data or financial data. Now you will see how to represent each word as vectors. There are various ways to vectorize text, such as:

* Words represented by each word as a vector
* Characters represented by each character as a vector
* N-grams of words/characters represented as a vector (N-grams are overlapping groups of multiple succeeding words/characters in the text)

Here, you’ll see how to deal with representing words as vectors which is the common way to use text in neural networks. Two possible ways to represent a word as a vector are one-hot encoding and word embeddings.

**One-Hot Encoding**

The first way to represent a word as a vector is by creating a so-called one-hot encoding, which is simply done by taking a vector of the length of the vocabulary with an entry for each word in the corpus.

In this way, you have for each word, given it has a spot in the vocabulary, a vector with zeros everywhere except for the corresponding spot for the word which is set to one.

In [78]:
cities = ['London', 'Berlin', 'Berlin', 'New York', 'London']
cities

['London', 'Berlin', 'Berlin', 'New York', 'London']

`LabelEncoder` to encode the list of cities into categorical integer values

OneHotEncoder expects each categorical value to be in a separate row, so you’ll need to reshape the array, then you can apply the encoder:

In [79]:
encoder = LabelEncoder()
city_labels = encoder.fit_transform(cities)
city_labels

array([1, 0, 0, 2, 1], dtype=int64)

In [83]:
encoder = OneHotEncoder(sparse=False)
city_labels = city_labels.reshape((5, 1))
encoder.fit_transform(city_labels)

C:\Users\7314315\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

**Word Embeddings**

This method represents words as dense word vectors (also called word embeddings) which are trained unlike the one-hot encoding which are hardcoded. This means that the word embeddings collect more information into fewer dimensions.

Note that the word embeddings do not understand the text as a human would, but they rather map the statistical structure of the language used in the corpus. Their aim is to map semantic meaning into a geometric space. This geometric space is then called the embedding space.

Now you need to tokenize the data into a format that can be used by the word embeddings. Keras offers a couple of convenience methods for text preprocessing and sequence preprocessing which you can employ to prepare your text.

You can start by using the Tokenizer utility class which can vectorize a text corpus into a list of integers. Each integer maps to a value in a dictionary that encodes the entire corpus, with the keys in the dictionary being the vocabulary terms themselves. You can add the parameter num_words, which is responsible for setting the size of the vocabulary. The most common num_words words will be then kept.

In [84]:
import nltk

In [85]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [87]:
#tokenizing : process of breaking down/tokenizing of an inpur sentence into words/tokens

In [88]:
s="today is friday and time is 2 o clock "

In [90]:
s.split(" ")

['today', 'is', 'friday', 'and', 'time', 'is', '2', 'o', 'clock', '']

In [109]:
import nltk
from nltk.corpus import stopwords
words=set(stopwords.words('english'))


In [111]:
clean_text =[x.split(" ") for x in sentences_train]

In [116]:
[w for w in clean_text[2] if not w in words] 

['I', 'fan', '...', 'This', 'movie', 'sucked', 'really', 'bad.', '', '']

In [117]:
cl_txt =[]
for i in range(len(clean_text)):
    cl_txt.append([w for w in clean_text[i] if not w in words] )
    
   

In [120]:
clean_text[2],cl_txt[2]

(['I',
  'am',
  'a',
  'fan',
  'of',
  'his',
  '...',
  'This',
  'movie',
  'sucked',
  'really',
  'bad.',
  '',
  ''],
 ['I', 'fan', '...', 'This', 'movie', 'sucked', 'really', 'bad.', '', ''])

In [135]:
cl_txt

[['Definitely', 'worth', 'checking', 'out.', '', ''],
 ['I', 'keep', 'watching', 'over.', '', ''],
 ['I', 'fan', '...', 'This', 'movie', 'sucked', 'really', 'bad.', '', ''],
 ['Lifetime',
  'air',
  'enough,',
  'anyone',
  'knows',
  'store',
  'sells',
  'let',
  'know',
  'must-have.',
  '',
  ''],
 ['Not', 'frightening', 'least,', 'barely', 'comprehensible.', '', ''],
 ['It',
  'year,',
  'however,',
  'reminded',
  'us',
  'Huston',
  'still',
  'top',
  'game',
  'evinced',
  'faithful',
  'adaptation',
  'James',
  "Joyce's",
  'acclaimed',
  'novella',
  '"The',
  'Dead.',
  '',
  ''],
 ['It', 'zillion', 'times', 'away', 'reality.', '', ''],
 ['Overall', 'I', 'rate', 'movie', '10', '1-10', 'scale.', '', ''],
 ['This', 'movie', 'mind-bendingly', 'awful,', 'even', 'created.', '', ''],
 ['Not',
  'sure',
  'lost',
  '-',
  'flat',
  'characters',
  'audience,',
  'nearly',
  'half',
  'walked',
  'out.',
  '',
  ''],
 ["It's", 'case', "'so", 'bad', "laughable'.", '', ''],
 ['This'

In [136]:
tokenizer = Tokenizer(num_words=5000)  # setting the vocab size
tokenizer.fit_on_texts(sentences_train)

In [137]:
tokenizer.word_index

{'the': 1,
 'a': 2,
 'and': 3,
 'of': 4,
 'is': 5,
 'this': 6,
 'i': 7,
 'it': 8,
 'to': 9,
 'in': 10,
 'movie': 11,
 'was': 12,
 'film': 13,
 'that': 14,
 '0': 15,
 'for': 16,
 'as': 17,
 '1': 18,
 'but': 19,
 'with': 20,
 'not': 21,
 'are': 22,
 'on': 23,
 "it's": 24,
 'one': 25,
 'you': 26,
 'all': 27,
 'so': 28,
 'just': 29,
 'bad': 30,
 'at': 31,
 'very': 32,
 'good': 33,
 'an': 34,
 'out': 35,
 'there': 36,
 'be': 37,
 'by': 38,
 'time': 39,
 'like': 40,
 'have': 41,
 'or': 42,
 'about': 43,
 'great': 44,
 'really': 45,
 'from': 46,
 'even': 47,
 'characters': 48,
 'his': 49,
 'who': 50,
 'if': 51,
 'more': 52,
 'see': 53,
 'acting': 54,
 'were': 55,
 '10': 56,
 'has': 57,
 'my': 58,
 'some': 59,
 'well': 60,
 'only': 61,
 'no': 62,
 'he': 63,
 'because': 64,
 'most': 65,
 'when': 66,
 "don't": 67,
 'its': 68,
 'how': 69,
 'story': 70,
 'movies': 71,
 'best': 72,
 'other': 73,
 "didn't": 74,
 'plot': 75,
 'character': 76,
 'can': 77,
 'also': 78,
 'than': 79,
 'real': 80,
 'up': 

In [138]:
sentences_train[0]

'Definitely worth checking out.  '

In [141]:
tokenizer.texts_to_sequences(sentences_train[0:2])

[[170, 116, 390, 35], [7, 310, 97, 8, 117, 3, 117]]

In [142]:
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

In [153]:
[len(x) for x in X_train]

[100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100

In [149]:
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(sentences_train[2])
print(X_train[2])

I am a fan of his ... This movie sucked really bad.  
[7, 150, 2, 932, 4, 49, 6, 11, 563, 45, 30]


In [ ]:
for word in ['the', 'all','fan']:
    print('{}: {}'.format(word, tokenizer.word_index[word]))

** pad sequences with Keras**

In [152]:
maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[23, :])

[ 31  62 321  10   1 960 171   8 114 585  40 961   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [156]:
X_train[0][0]

170

In [168]:
sentences_train[76]

'However, after seeing the short again after about 25 years, I was amazed at how timeless the film actually is.  '

In [ ]:
s1 = 'happiness is not expensive but happy joy and contentment'

In [159]:
sentences_train[5]

'It was that year, however, that reminded us that Huston was still at the top of his game as evinced by his faithful adaptation of James Joyce\'s acclaimed novella "The Dead.  '

### Keras Embedding Layer

Now you can use the Embedding Layer of Keras which takes the previously calculated integers and maps them to a dense vector of the embedding. You will need the following parameters:
* **input_dim**: the size of the vocabulary
* **output_dim**: the size of the dense vector
* **input_length**: the length of the sequence

With the Embedding layer we have now a couple of options. One way would be to take the output of the embedding layer and plug it into a Dense layer. In order to do this you have to add a Flatten layer in between that prepares the sequential input for the Dense layer:

In [174]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))


model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 50)           128750    
_________________________________________________________________
flatten_4 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 100)               500100    
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 101       
Total params: 628,951
Trainable params: 628,951
Non-trainable params: 0
_________________________________________________________________


In [175]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

Train on 561 samples, validate on 187 samples
Epoch 1/20
561/561 [==============================] - 1s 2ms/step - loss: 0.6975 - accuracy: 0.5009 - val_loss: 0.6910 - val_accuracy: 0.4920
Epoch 2/20
561/561 [==============================] - 1s 2ms/step - loss: 0.6217 - accuracy: 0.7291 - val_loss: 0.6608 - val_accuracy: 0.6257
Epoch 3/20
561/561 [==============================] - 1s 2ms/step - loss: 0.3558 - accuracy: 0.9251 - val_loss: 0.7164 - val_accuracy: 0.5722
Epoch 4/20
561/561 [==============================] - 1s 2ms/step - loss: 0.1177 - accuracy: 0.9875 - val_loss: 0.6253 - val_accuracy: 0.7112
Epoch 5/20
561/561 [==============================] - 1s 1ms/step - loss: 0.0472 - accuracy: 0.9929 - val_loss: 0.7217 - val_accuracy: 0.6898
Epoch 6/20
561/561 [==============================] - 1s 2ms/step - loss: 0.0224 - accuracy: 1.0000 - val_loss: 0.6799 - val_accuracy: 0.7219
Epoch 7/20
561/561 [==============================] - ETA: 0s - loss: 0.0120 - accuracy: 1.00 - 1s 2ms

KeyError: 'acc'

Global max/average pooling takes the maximum/average of all features whereas in the other case you have to define the pool size. Keras has again its own layer that you can add in the sequential model:

In [ ]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

## Convolutional Neural Networks (CNN)

Convolutional neural networks or also called **convnets** are one of the most exciting developments in machine learning in recent years.

They have revolutionized image classification and computer vision by being able to extract features from images and using them in neural networks. The properties that made them useful in image processing makes them also handy for sequence processing. You can imagine a CNN as a specialized neural network that is able to detect specific patterns.

A CNN has hidden layers which are called convolutional layers. When you think of images, a computer has to deal with a two dimensional matrix of numbers and therefore you need some way to detect features in this matrix. These convolutional layers are able to detect edges, corners and other kinds of textures which makes them such a special tool. The convolutional layer consists of multiple filters which are slid across the image and are able to detect specific features.

In [ ]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

## Hyperparameters Optimization

One crucial steps of deep learning and working with neural networks is hyperparameter optimization.

As you saw in the models that we have used so far, even with simpler ones, you had a large number of parameters to tweak and choose from. Those parameters are called hyperparameters. This is the most time consuming part of machine learning and sadly there are no one-fits-all solutions ready.

One popular method for hyperparameter optimization is **grid search**. What this method does is it takes lists of parameters and it runs the model with each parameter combination that it can find. It is the most thorough way but also the most computationally heavy way to do this. Another common way,**random search**, which you’ll see in action here, simply takes random combinations of parameters.

In order to apply random search with Keras, you will need to use the KerasClassifier which serves as a wrapper for the scikit-learn API. With this wrapper you are able to use the various tools available with scikit-learn like cross-validation. The class that you need is RandomizedSearchCV which implements random search with cross-validation. Cross-validation is a way to validate the model and take the whole data set and separate it into multiple testing and training data sets.

There are various types of cross-validation. One type is the k-fold cross-validation. In this type the data set is partitioned into k equal sized sets where one set is used for testing and the rest of the partitions are used for training. This enables you to run k different runs, where each partition is once used as a testing set. So, the higher k is the more accurate the model evaluation is, but the smaller each testing set is.

In [ ]:
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size=[5000], 
                  embedding_dim=[50],
                  maxlen=[100])

In [ ]:
# Main settings
epochs = 20
embedding_dim = 50
maxlen = 100
output_file = 'output.txt'

# Run grid search for each source (yelp, amazon, imdb)
for source, frame in df.groupby('source'):
    print('Running grid search for data set :', source)
    sentences = df['sentence'].values
    y = df['label'].values

    # Train-test split
    sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentences, y, test_size=0.25, random_state=1000)

    # Tokenize words
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)
    X_train = tokenizer.texts_to_sequences(sentences_train)
    X_test = tokenizer.texts_to_sequences(sentences_test)

    # Adding 1 because of reserved 0 index
    vocab_size = len(tokenizer.word_index) + 1

    # Pad sequences with zeros
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

    # Parameter grid for grid search
    param_grid = dict(num_filters=[32, 64, 128],
                      kernel_size=[3, 5, 7],
                      vocab_size=[vocab_size],
                      embedding_dim=[embedding_dim],
                      maxlen=[maxlen])
    model = KerasClassifier(build_fn=create_model,
                            epochs=epochs, batch_size=10,
                            verbose=False)
    grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=1, n_iter=5)
    grid_result = grid.fit(X_train, y_train)

    # Evaluate testing set
    test_accuracy = grid.score(X_test, y_test)

    # Save and evaluate results
#     prompt = input(f'finished {source}; write to file and proceed? [y/n]')
#     if prompt.lower() not in {'y', 'true', 'yes'}:
#         break
#     with open(output_file, 'w+') as f:
    s = ('Running {} data set\nBest Accuracy : '
             '{:.4f}\n{}\nTest Accuracy : {:.4f}\n\n')
    output_string = s.format(
        source,
        grid_result.best_score_,
        grid_result.best_params_,
        test_accuracy)
    print(output_string)
#         f.write(output_string)